<a href="https://colab.research.google.com/github/Mudassir4545/Satellite-Positioning-using-KNN/blob/main/Knn_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
# Importing packages - Pandas, Numpy, Seaborn, Scipy
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, sys
import matplotlib.style as style; style.use('fivethirtyeight')
from scipy.stats import zscore, norm
from math import sqrt
from random import seed
from random import randrange
from sklearn import preprocessing
# Modelling - LR, KNN, NB, Metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
import glob
import csv

## load dataset Txt file

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive"

In [4]:
TIME = np.loadtxt(path+'/MLearning_input_TIME_1yr.txt')
AE = np.loadtxt(path+'/MLearning_input_AE_1yr.txt')
AL = np.loadtxt(path+'/MLearning_input_AL_1yr.txt')
AU = np.loadtxt(path+'/MLearning_input_AU_1yr.txt')
AP = np.loadtxt(path+'/MLearning_input_AP_1yr.txt')
KP = np.loadtxt(path+'/MLearning_input_KP_1yr.txt')
BX = np.loadtxt(path+'/MLearning_input_BX_1yr.txt')
BY = np.loadtxt(path+'/MLearning_input_BY_1yr.txt')
BZ = np.loadtxt(path+'/MLearning_input_BZ_1yr.txt')
VX = np.loadtxt(path+'/MLearning_input_VX_1yr.txt')
VY = np.loadtxt(path+'/MLearning_input_VY_1yr.txt')
VZ = np.loadtxt(path+'/MLearning_input_VZ_1yr.txt')
SYMH = np.loadtxt(path+'/MLearning_input_SYMH_1yr.txt')
F107 = np.loadtxt(path+'/MLearning_input_F107_1yr.txt')
#DOY = np.loadtxt(path+'/MLearning_input_DOY_1yr.txt')
COORD = np.loadtxt(path+'/MLearning_input_COORD_1yr.txt')
DEN =np.loadtxt(path+'/MLearning_input_DEN_1yr.txt')


In [5]:
XTIME = TIME[:,0:1] # 35
XAP = AP[:,0:1]
XKP = KP[:,0:1]
XBX = BX[:,0:1]
XBY = BY[:,0:1]
XBZ = BZ[:,0:1]
XVX = VX[:,0:1]
XVY = VY[:,0:1]
XVZ = VZ[:,0:1] 
XSYMH = SYMH[:,0:1]# 27
XF107 = F107[:,0:1]
#XDOY = DOY[:,0:1] # 33
XCOORD = COORD[:,0:1] # Geog. latitude, COS (GLONG), SIN (GLONG) # 53
XDEN= DEN




In [6]:
lst = [XTIME,XAP,XKP,XBX,XBY,XBZ,XVX,XVY,XVZ,XSYMH,XF107,XCOORD,XDEN]

In [7]:
dataset = np.column_stack((XTIME,XAP,XKP,XBX,XBY,XBZ,XVX,XVY,XVZ,XSYMH,XF107,XCOORD,XDEN)) ## create your data with name dataset = [column1, column2, ... column57]
# split into input (X) and output (Y) variables
dataset = dataset[~np.isnan(dataset).any(axis=1),:]

#new_data = dataset[np.logical_not(np.isnan(dataset))]

In [8]:
dataset.shape

(2642153, 13)

In [9]:
df = pd.DataFrame(dataset, columns = ["XTIME",'XAP','XKP','XBX','XBY','XBZ','XVX','XVY','XVZ','XSYMH','XF107','XCOORD','XDEN'])

In [10]:
df.head()

,XTIME,XAP,XKP,XBX,XBY,XBZ,XVX,XVY,XVZ,XSYMH,XF107,XCOORD,XDEN
0,0.000000,2.500000,0.500000,-2.940000,3.630000,-0.680000,-319.70000,-7.700000,12.100000,0.0,171.6,-6.712439,1.053846e-12
1,0.002778,2.505556,0.501856,-2.941667,3.627333,-0.680667,-319.67667,-7.636667,12.096667,0.0,171.6,-7.350848,1.054047e-12
2,0.005556,2.511111,0.503711,-2.943333,3.624667,-0.681333,-319.65333,-7.573333,12.093333,0.0,171.6,-7.989212,1.057566e-12
3,0.008333,2.516667,0.505567,-2.945000,3.622000,-0.682000,-319.63000,-7.510000,12.090000,0.0,171.6,-8.627530,1.043966e-12
4,0.011111,2.522222,0.507422,-2.946667,3.619333,-0.682667,-319.60667,-7.446667,12.086667,0.0,171.6,-9.265799,1.048014e-12


In [11]:
df.tail()

,XTIME,XAP,XKP,XBX,XBY,XBZ,XVX,XVY,XVZ,XSYMH,XF107,XCOORD,XDEN
2642148,23.986111,4.944444,1.314972,5.236667,-2.225,1.781667,-531.36667,-11.85,-11.383333,-18.0,129.2,54.591505,1.443914e-12
2642149,23.988889,4.955556,1.318678,5.195333,-2.250,1.765333,-531.33333,-11.44,-11.546667,-18.4,129.2,53.953853,1.462110e-12
2642150,23.991667,4.966667,1.322383,5.154000,-2.275,1.749000,-531.30000,-11.03,-11.710000,-18.8,129.2,53.316064,1.465375e-12
2642151,23.994444,4.977778,1.326089,5.112667,-2.300,1.732667,-531.26667,-10.62,-11.873333,-19.2,129.2,52.678140,1.461143e-12
2642152,23.997222,4.988889,1.329794,5.071333,-2.325,1.716333,-531.23333,-10.21,-12.036667,-19.6,129.2,52.040085,1.465039e-12


In [12]:
df['XTIME']=df['XTIME']/df['XTIME'].max()
df['XAP']=df['XAP']/df['XAP'].max()
df['XKP']=df['XKP']/df['XKP'].max()
df['XBX']=df['XBX']/df['XBX'].max()
df['XBY']=df['XBY']/df['XBY'].max()
df['XBZ']=df['XBZ']/df['XBZ'].max()
df['XVX']=df['XVX']/df['XVX'].max()
df['XVY']=df['XVY']/df['XVY'].max()
df['XVZ']=df['XVZ']/df['XVZ'].max()
df['XSYMH']=df['XSYMH']/df['XSYMH'].max()
df['XF107']=df['XF107']/df['XF107'].max()
df['XCOORD']=df['XCOORD']/df['XCOORD'].max()
df['XDEN']=df['XDEN']/df['XDEN'].max()

In [13]:
df.dtypes

XTIME     float64
XAP       float64
XKP       float64
XBX       float64
XBY       float64
XBZ       float64
XVX       float64
XVY       float64
XVZ       float64
XSYMH     float64
XF107     float64
XCOORD    float64
XDEN      float64
dtype: object

In [14]:
# Checking for missing values
df.isnull().sum()

XTIME     0
XAP       0
XKP       0
XBX       0
XBY       0
XBZ       0
XVX       0
XVY       0
XVZ       0
XSYMH     0
XF107     0
XCOORD    0
XDEN      0
dtype: int64

In [15]:
df.dropna().isnull().sum()

XTIME     0
XAP       0
XKP       0
XBX       0
XBY       0
XBZ       0
XVX       0
XVY       0
XVZ       0
XSYMH     0
XF107     0
XCOORD    0
XDEN      0
dtype: int64

In [16]:
df.corr().style.background_gradient(cmap='coolwarm')

,XTIME,XAP,XKP,XBX,XBY,XBZ,XVX,XVY,XVZ,XSYMH,XF107,XCOORD,XDEN
XTIME,1.000000,-0.018045,-0.019917,0.019374,-0.010171,0.010352,0.004226,-0.011725,-0.015105,0.038979,-0.001436,0.000190,-0.003106
XAP,-0.018045,1.000000,0.887475,-0.004871,0.022593,-0.202889,0.396575,0.082128,-0.039848,-0.418114,0.088056,0.000865,0.197902
XKP,-0.019917,0.887475,1.000000,-0.034378,0.024932,-0.238693,0.456693,0.128727,-0.033459,-0.428059,0.136231,0.001767,0.239452
XBX,0.019374,-0.004871,-0.034378,1.000000,-0.439562,0.170707,0.125853,0.092162,0.023782,0.079272,-0.191630,-0.002524,0.004085
XBY,-0.010171,0.022593,0.024932,-0.439562,1.000000,-0.069848,-0.055866,-0.104343,-0.043834,-0.115144,0.159644,0.002405,0.022063
XBZ,0.010352,-0.202889,-0.238693,0.170707,-0.069848,1.000000,0.058866,0.023061,-0.042934,0.240893,-0.015366,0.001040,-0.010781
XVX,0.004226,0.396575,0.456693,0.125853,-0.055866,0.058866,1.000000,0.296560,-0.024418,-0.248624,0.140654,-0.000906,0.286016
XVY,-0.011725,0.082128,0.128727,0.092162,-0.104343,0.023061,0.296560,1.000000,-0.036980,-0.076450,0.009283,-0.001822,0.071535
XVZ,-0.015105,-0.039848,-0.033459,0.023782,-0.043834,-0.042934,-0.024418,-0.036980,1.000000,-0.009374,0.060513,0.000604,0.018250
XSYMH,0.038979,-0.418114,-0.428059,0.079272,-0.115144,0.240893,-0.248624,-0.076450,-0.009374,1.000000,-0.105987,-0.001548,-0.199587


In [ ]:
# Let's see the possible best variables for modeling the carat
sns.pairplot(df);

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df['XDEN'].isnull().values.any()

In [ ]:
# Split dataset into input features and label
X = dataset[:,0:12]   
Y = dataset[:,12]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size= 0.2, random_state=12)

In [ ]:
print(f'Train: {X_train.shape, y_train.shape}  \nTest: {X_test.shape, y_test.shape}')

In [ ]:
# Extract the descriptive statistics for the explanatory variables for IQR calculation
df.describe()

In [ ]:
df

In [ ]:
# setup figure
#fig, ax = plt.subplots(1,3, figsize=(20,7))

# plot
g1 = sns.boxplot(y=X_train, color='green')
g1.set_title('Boxplot of x_train')


In [ ]:
# plot
g1 = sns.boxplot(y=X_test, color='blue')
g1.set_title('Boxplot of x_test')

In [ ]:
# Log transformation
X_log = np.log(X_train)
y_log = np.log(y_train)

In [ ]:
dataset =df.values

In [ ]:
dataset

In [ ]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

In [ ]:
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

In [ ]:
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax
 

In [ ]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for _ in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 


In [ ]:

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

## Calculate Euclidean Distance

In [ ]:
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

In [ ]:
row0 = dataset[0]
for row in dataset:
	distance = euclidean_distance(row0, row)
	#print(distance)

## Get Nearest Neighbours

In [ ]:
# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

## Test Distance function

In [ ]:
neighbors = get_neighbors(dataset, dataset[0], 13)
count = 0
for neighbor in neighbors:
	count+=1
	print(neighbor)
print(count)

In [ ]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 


In [ ]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for _ in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [ ]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [ ]:
# kNN Algorithm
def k_nearest_neighbors(X_train, X_test, num_neighbors):
	predictions = list()
	for row in X_test:
		output = predict_classification(X_train, row, num_neighbors)
		predictions.append(output)
	return(predictions)

## Make Prediction

In [ ]:
# Make a classification prediction with neighbors
def predict_classification(X_train, test_row, num_neighbors):
	neighbors = get_neighbors(X_train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction

In [ ]:
prediction = predict_classification(dataset, dataset[0], 13)
print('Expected %d, Got %d.' % (dataset[0][-2]*(-100), prediction*100))


In [ ]:
# evaluate algorithm
n_folds = 2000
num_neighbors = 13
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))